In [0]:
# import necessary libraries

import re
import os
import json
import nltk
import pandas as pd
from collections import OrderedDict 
from nltk import sent_tokenize 
from IPython.display import clear_output
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import numpy as np
import pickle

nltk.download('all')

In [0]:
# Regex format matching dictionary
dict_regex = {}
#start and end
dict_regex['hh:mm:ss.ss ra'] = re.compile('\d{1,2}:\d{2}:\d{2}\.?\d*')
dict_regex['dd:mm:ss.ss dec'] = re.compile('[+-]?\d{1,2}:\d{2}:\d{2}\.?\d*')

dict_regex['hh mm ss.ss ra'] = re.compile('\d{1,2}\s\d{2}\s\d{2}\.?\d*')
dict_regex['dd mm ss.ss dec'] = re.compile('[+-]?\d{1,2}\s\d{2}\s\d{2}\.?\d*')

dict_regex['00h 00m 00.0s ra'] = re.compile('\d{1,2}h\s?\d{2}m\s?\d{2}\.?\d*s?')
dict_regex['00d 00m 00.0s dec'] = re.compile('[+-]?\d{1,2}d\s?\d{2}m\s?\d{2}\.?\d*s?')

dict_regex['dd:mm:ss.ss ra'] = re.compile('\d{1,3}:\d{2}:\d{2}\.?\d*')
dict_regex['dd mm ss.ss ra'] = re.compile('\d{1,3}\s\d{2}\s\d{2}\.?\d*')
dict_regex['00d 00m 00.0s ra'] = re.compile('\d{1,3}d\s?\d{2}m\s?\d{2}\.?\d*s?')

dict_regex['00do 00\' 00.0\" dec'] = re.compile('[+-]?\d{1,2}[do]\s?\d{2}\'\s?\d{2}\.?\d*(\"|\'\'|”)')

list_coords_regex = []

list_coords_regex.append((dict_regex['hh:mm:ss.ss ra'], dict_regex['dd:mm:ss.ss dec']))
list_coords_regex.append((dict_regex['hh mm ss.ss ra'], dict_regex['dd mm ss.ss dec']))
list_coords_regex.append((dict_regex['00h 00m 00.0s ra'], dict_regex['00d 00m 00.0s dec']))
list_coords_regex.append((dict_regex['dd:mm:ss.ss ra'], dict_regex['dd:mm:ss.ss dec']))
list_coords_regex.append((dict_regex['dd mm ss.ss ra'], dict_regex['dd mm ss.ss dec']))
list_coords_regex.append((dict_regex['00d 00m 00.0s ra'], dict_regex['00d 00m 00.0s dec']))
list_coords_regex.append((dict_regex['00h 00m 00.0s ra'], dict_regex['00do 00\' 00.0\" dec']))
list_coords_regex.append((dict_regex['00d 00m 00.0s ra'], dict_regex['00do 00\' 00.0\" dec']))

In [0]:
def get_coordinates_except_decimal_sentences(sentence, start_index, lst_coords):

    for coord_regex in list_coords_regex:
    
        ra_search_obj = coord_regex[0].search(sentence[start_index:])
    
        if ra_search_obj:
            ra_end_ind = ra_search_obj.end()
            dec_search_obj = coord_regex[1].search(sentence[start_index+ra_end_ind:])
            if dec_search_obj:
                lst_coords.append((ra_search_obj.group(), dec_search_obj.group()) )
                start_index_next = start_index + ( ra_end_ind + dec_search_obj.end() )
                get_coordinates_except_decimal_sentences(sentence, start_index_next, lst_coords)
            break
        else:
            continue
      

def get_coordinates_except_decimal(content):
  lst = []
  for sentence in sent_tokenize(content):
    get_coordinates_except_decimal_sentences(sentence, 0, lst)
  return lst

In [0]:
# Sentence Tokenizer

def my_tokenizer(input_string):         # UPDATED
    sentences = sent_tokenize(input_string)
    flag_idices = []
    sentences_new = []
    for sent_idx in range(0,len(sentences)):
        if( sent_idx in flag_idices ):
            continue
        temp_str = sentences[sent_idx]
        temp_idx = sent_idx
        tuple_end = ("Decl.", "et al.", 'e.g.', 'Coll.')
        while( sentences[temp_idx].endswith(tuple_end) ):
            flag_idices.append(temp_idx+1)
            if(temp_idx+1 >= len(sentences)):
                break
            temp_str += sentences[temp_idx+1]
            temp_idx+=1
        
        sentences_new.append(temp_str)

    return sentences_new


def print_sentence(input_string):
        t_len = 0
        for j in range(len(input_string)):
            print(input_string[j], end='')
            t_len+=1
            if(t_len%100==0):
                print('\n')
        print("\n")



In [0]:
# Some Preprocessing

def process_sent(sentence):
    #stemmer object
    ps = PorterStemmer()
    #preserving the J2000 keyword 
    review = re.sub('J2000', 'jtwothousand', sentence)
    #replacing all charactars except alphabets by space
    review = re.sub('[^a-zA-Z\.]', ' ', review)
    #removing the single ' . '
    review = re.sub('\s\.\s', ' ', review)
    #lowering all charactars 
    review = review.lower()
    #splitting along the spaces
    review = review.split()
    #removing '.' from words
    review = [re.sub('\.', '', word) for word in review]
    #stemming and removing stop words
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    #joining the words to a sentence
    review = ' '.join(review)
    review = re.sub('decl', 'dec', review)
    if len(review)==0:
        return review
    #removing trailing '.'
    elif review[-1] == '.':
        review = review[:-1]
    return review


In [0]:
#extract object name format

def get_object_name_format(sentence):
    #filtering by removing sentences with the keyword grism
    if re.search('grism', sentence.lower()):
        return []
    lst_match = re.findall(r'\d+\.?\d*[\+-]\d+\.?\d*', sentence) 
    lst_output = []
    #adding matches only for cases length of ra is [4,6]
    #and length of dec is [3,6]
    for str_match in lst_match:
        ra, dec = re.split('\+|-', str_match)
        ra_int = ra.split('.')[0]
        dec_int = dec.split('.')[0]
        if (4<=len(ra_int)<=6) and (3<=len(dec_int)<=6):
            lst_output.append(str_match)
    return lst_output

#extract decimal coordinates

def get_decimal_coordinates_sentences(sentence, start_index, lst_coords):

    coord_regex = re.compile('\D\d{1,3}\.\d+'), re.compile('\D[+-]?\d{1,2}\.\d+')

    ra_search_obj = coord_regex[0].search(sentence[start_index:])

    if ra_search_obj:
        ra_end_ind = ra_search_obj.end()
        dec_search_obj = coord_regex[1].search(sentence[start_index+ra_end_ind:])
        if dec_search_obj:
            lst_coords.append((ra_search_obj.group(), dec_search_obj.group() ))
        start_index_next = start_index + ra_end_ind 
        get_decimal_coordinates_sentences(sentence, start_index_next, lst_coords)

def get_decimal_coordinates(content):
  lst = []
  for sentence in sent_tokenize(content):
    get_decimal_coordinates_sentences(sentence, 0, lst)
  return lst


In [0]:
# Load Model

filename = '/content/drive/My Drive/StarWords/model.pickle'
model = pickle.load(open(filename, "rb"))


In [0]:
# INFERENCE

def classify(x_test_sentence):
    x_test_processed = process_sent(x_test_sentence)
    y_pred_test = model.predict([x_test_processed])
    
    return y_pred_test

In [0]:
def extract_all_coordinates(input_string):
    
    all_matches = []

    for sentence in my_tokenizer(input_string):

        print_sentence(sentence)
        label = classify(sentence) #done
        
        object_name_formats = get_object_name_format(sentence)
        all_matches.extend(object_name_formats) #done
        # print(f'object name formats are {object_name_formats}')

        matches = get_coordinates_except_decimal(sentence) # done
        # print(f'matches except decimal are {matches}')
        if label == 1:
            # print('label = ', label)
            if len(matches)==0:
                decimal_format = get_decimal_coordinates(sentence)
                # print(f'decimal format are {decimal_format}')
                matches.extend(decimal_format)  #to do
            all_matches.extend(matches)
        # print('-----------')


    return set(all_matches)